In [1]:
import json
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root = "/content/drive/MyDrive/CV Project 2"

In [12]:
image_folder = os.path.join(root, 'images')

In [4]:
# Read the JSON file
with open(os.path.join(root, "groundtruth.json"), 'r') as file:
    json_data = file.read()

In [5]:
# Convert JSON to Python dictionary
dictionary = json.loads(json_data)

In [6]:
for key, value in dictionary.items():
    print(f"Key: {key}, Value: {value}")

Key: 0, Value: {'query': '100000.jpg', 'similar': ['100001.jpg', '100002.jpg']}
Key: 1, Value: {'query': '100100.jpg', 'similar': ['100101.jpg']}
Key: 2, Value: {'query': '100200.jpg', 'similar': ['100201.jpg']}
Key: 3, Value: {'query': '100300.jpg', 'similar': ['100301.jpg', '100302.jpg']}
Key: 4, Value: {'query': '100400.jpg', 'similar': ['100401.jpg']}
Key: 5, Value: {'query': '100500.jpg', 'similar': ['100501.jpg', '100502.jpg', '100503.jpg']}
Key: 6, Value: {'query': '100600.jpg', 'similar': ['100601.jpg']}
Key: 7, Value: {'query': '100700.jpg', 'similar': ['100701.jpg']}
Key: 8, Value: {'query': '100800.jpg', 'similar': ['100801.jpg']}
Key: 9, Value: {'query': '100900.jpg', 'similar': ['100901.jpg', '100902.jpg']}
Key: 10, Value: {'query': '101000.jpg', 'similar': ['101001.jpg']}
Key: 11, Value: {'query': '101100.jpg', 'similar': ['101101.jpg']}
Key: 12, Value: {'query': '101200.jpg', 'similar': ['101201.jpg']}
Key: 13, Value: {'query': '101300.jpg', 'similar': ['101301.jpg']}
Ke

In [7]:
len(os.listdir(os.path.join(root, 'images')))

1491

In [9]:
def find_similar_images(query_image_path, image_names, num_similar_images=4):
    # Load query image
    query_image = cv2.imread(os.path.join(root, query_image_path), cv2.IMREAD_GRAYSCALE)
    query_image = cv2.resize(query_image, (128, 128))

    surf = cv2.xfeatures2d.SURF_create()

    query_keypoints, query_descriptors = surf.detectAndCompute(query_image, None)

    flann = cv2.FlannBasedMatcher()

    matching_scores = []
    matching_image_paths = []

    for img_name in image_names:

        image_path = os.path.join(image_folder, img_name)

        # Load image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 128))

        image_keypoints, image_descriptors = surf.detectAndCompute(image, None)

        matches = flann.knnMatch(query_descriptors, image_descriptors, k=2)

        # Apply ratio test to filter good matches
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        # Store the number of good matches
        matching_scores.append(len(good_matches))
        matching_image_paths.append(image_path)

    # Sort the matching scores and corresponding image paths
    sorted_indices = np.argsort(matching_scores)[::-1]
    sorted_scores = np.array(matching_scores)[sorted_indices]
    sorted_image_paths = np.array(matching_image_paths)[sorted_indices]

    # Return top similar images
    top_similar_images = sorted_image_paths[:num_similar_images]

    return top_similar_images

In [11]:
output = {}

for key in dictionary.keys():
    value = dictionary[key]
    output[key] = find_similar_images(value['query'], value['similar'], num_similar_images=4)

In [13]:
with open(os.path.join(root, 'surf_output.json'), 'w') as json_file:
    json.dump(output, json_file)